**CSC-555**

**Data Preprocessing**

In [1]:
import pyspark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1731866518661_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, when, col, avg, countDistinct

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Restaurants Test") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# S3 Bucket and File Paths
s3_bucket = "s3://bucket-akshat-test"
restaurants_path = f"{s3_bucket}/Restaurants/restaurants.csv"
menus_path = f"{s3_bucket}/Restaurants/restaurant-menus.csv"
cleaned_restaurants_path = f"{s3_bucket}/cleaned/restaurants/"
cleaned_menus_path = f"{s3_bucket}/cleaned/menus/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Load Dataset
restaurants_df = spark.read.csv(restaurants_path, header=True, inferSchema=True)
menus_df = spark.read.csv(menus_path, header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Data Cleaning 
- Restaurants

Restaurants Data:
- Normalizes price_range to numeric values.
- Standardizes category using a slash-separated format.

In [ ]:
restaurants_cleaned = restaurants_df \
    .withColumn("price_range", when(col("price_range") == "$", 1)
                                .when(col("price_range") == "$$", 2)
                                .when(col("price_range") == "$$$", 3)
                                .when(col("price_range") == "$$$$", 4)
                                .otherwise(None)) \
    .withColumn("category", regexp_replace(col("category"), ", ", "/"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

- Menus


Menus Data:
- Strips "USD" from price and converts it to a float.
- Replaces null description with "No description available."
- Normalizes category (e.g., removes redundant prefixes like "Extra Large").
- Sets default price to 0.0 for missing values.

In [ ]:
menus_cleaned = menus_df \
    .withColumn("price", regexp_replace(col("price"), " USD", "").cast("float")) \
    .withColumn("description", when(col("description").isNull(), "No description available")
                                .otherwise(col("description"))) \
    .withColumn("category", regexp_replace(col("category"), "(Extra Large|Jumbo|Spicy) ", "")) \
    .withColumn("price", when(col("price").isNull(), 0.0).otherwise(col("price")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Saving Cleaned data to S3
restaurants_cleaned.write.mode("overwrite").parquet(cleaned_restaurants_path)
menus_cleaned.write.mode("overwrite").parquet(cleaned_menus_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
from pyspark.sql.types import IntegerType, DoubleType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
restaurants_cleaned.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- position: string (nullable = true)
 |-- name: string (nullable = true)
 |-- score: string (nullable = true)
 |-- ratings: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price_range: integer (nullable = true)
 |-- full_address: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)

In [18]:
restaurants_cleaned = restaurants_cleaned.withColumn("id", restaurants_cleaned["id"].cast(IntegerType())) \
                                         .withColumn("position", restaurants_cleaned["position"].cast(IntegerType())) \
                                         .withColumn("ratings", restaurants_cleaned["ratings"].cast(IntegerType())) \
                                         .withColumn("lat", restaurants_cleaned["lat"].cast(DoubleType())) \
                                         .withColumn("lng", restaurants_cleaned["lng"].cast(DoubleType()))

restaurants_cleaned.write.mode("overwrite").parquet(cleaned_restaurants_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
menus_cleaned.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- restaurant_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- price: double (nullable = true)

In [20]:
menus_cleaned = menus_cleaned.withColumn("restaurant_id", menus_cleaned["restaurant_id"].cast(IntegerType()))
menus_cleaned.write.mode("overwrite").parquet(cleaned_menus_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Verify Cleaning
restaurants_cleaned.show(5)
menus_cleaned.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------+--------------------+-----+-------+--------------------+-----------+--------------------+--------+----------+-----------+
| id|position|                name|score|ratings|            category|price_range|        full_address|zip_code|       lat|        lng|
+---+--------+--------------------+-----+-------+--------------------+-----------+--------------------+--------+----------+-----------+
|  1|      19|PJ Fresh (224 Dan...| NULL|   NULL|Burgers/American/...|          1|224 Daniel Payne ...|   35207|33.5623653|-86.8307025|
|  2|       9|J' ti`'z Smoothie...| NULL|   NULL|Coffee and Tea/Br...|       NULL|1521 Pinson Valle...|   35217|  33.58364|  -86.77333|
|  3|       6|Philly Fresh Chee...| NULL|   NULL|American/Cheesest...|          1|541-B Graymont Av...|   35204|   33.5098|  -86.85464|
|  4|      17|Papa Murphy's (15...| NULL|   NULL|               Pizza|          1|1580 Montgomery H...|   35226|33.4044388|-86.8066142|
|  5|     162|Nelson Brothers C...|  4.7|     22

In [23]:
# Stop Spark Session
spark.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…